# Food vision

## Check GPU

* Mixed precision training: mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory.

The GPU we use with **mixed precision training** need to have compute capability 7.0 or above

The standred GPUs from colab is **Tesla P100** which have 6.0 compute capability, **K80** which have 3.7 and **Tesla T4** which have 7.0 capability.

NVIDIA A100 =>8.0

NVIDIA P4   =>7.5

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-0161f4af-8b69-12d3-e17b-d771b9a7e34c)


## Get helper functions

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-04-01 22:25:17--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-04-01 22:25:17 (78.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# Import sereis of helper functions
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Use Tensorflow Datasetes to download datasete

Tensorflow Datasets: https://www.tensorflow.org/datasets

Food 101 https://www.tensorflow.org/datasets/catalog/food101

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
datasets_list = tfds.list_builders()
print('food101' in datasets_list)

True


In [ ]:
# load in food101 dataset
(train_data, test_data), ds_info = tfds.load(name='food101',
                                             split=['train','validation'],
                                             shuffle_files=True,
                                             as_supervised=True, # Data will be in tupel format (data,label)
                                             with_info=True) #Will download the metadata as well

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

## Exploring the food101 dataset

* class names
* The shape of our input data(image tensor)
* The data type of out input data
* What the label look like (e.g. Are thay one hot encoded or label encoded)
* Do the label match up with class names

In [ ]:
##Features of the data
ds_info.features

In [ ]:
class_names = ds_info.features['label'].names
class_names[:10] #first 10

In [ ]:
train_one_sample = train_data.take(1) # take one sample of `train_data`
                                      # sample in format (image_tensor, label)
train_one_sample

In [ ]:
for image, label in train_one_sample:
  print(f"""
  image shape: {image.shape}
  image datatypr: {image.dtype}
  image label: {label}
  class name: {class_names[label.numpy()]}
  """)

In [ ]:
# What does our image look like
image

In [ ]:
tf.reduce_min(image), tf.reduce_max(image)

## Plot an image from the Dataset

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False);

## Preprocess out data

we need to:
1. Reshape our images to the same size.
2. Convert the dtypr=e of our images form `unit8` to `float32`.

In [ ]:
def preprocess_img(image, label, img_shape=256):
  image = tf.image.resize(images=image, size=[img_shape, img_shape]) #resize the image
  #image = image/255 #We use this line in model dont have scaling function built in
  return tf.cast(image, tf.float32), label #cast the image to new dtype

In [ ]:
# try the function
preprocessed_img = preprocess_img(image, label)[0] #0 to print the image
print(f"Model befor preprocessing\n{image[:2]}....\nImage shape {image.shape}\nImage data type {image.dtype}")
print(f"Model after preprocessing\n{preprocessed_img[:2]}\nImage shape {preprocessed_img.shape}\nImage data type {preprocessed_img.dtype}")

## Batch the Dataset

In [ ]:
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
#Shuffel data and turn it to batches
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)
# Map preprocessing function to the test data
test_data = test_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_data, test_data

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

## Create modelling callbacks
* TensorBoard callback
* ModelCheckpoint callback

In [ ]:
#creat tensorboared callback
from helper_functions import create_tensorboard_callback

#Create model check point callback
ModelCheckPoint = tf.keras.callbacks.ModelCheckpoint(filepath='model_checkpoints/cp.ckpt',
                                                     monitor='val_acc',
                                                     save_best_only=True,
                                                     save_weights_only=True,
                                                     verbose=0)

In [ ]:
tf.__version__

'2.12.0'

## Setup mixed precision training

https://www.tensorflow.org/guide/mixed_precision

The main goal of it to speed up the model performance

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

## The model body

In [ ]:
#base model
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable= False

# functional model
Input = tf.keras.layers.Input(shape=(256,256,3), name='input_layer')
x = base_model(Input, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name='pooling_layer')(x)
x = tf.keras.layers.Dense(len(class_names))(x)
Output = tf.keras.layers.Activation('softmax', dtype=tf.float32)(x)
model = tf.keras.Model(Input, Output)
#compile
model.compile(loss = 'sparse_categorical_crossentropy', #When the label integers(not hot-encoded) we use the sparse
              optimizer = tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 pooling_layer (GlobalAverag  (None, 1280)             0         
 ePooling2D)                                                     
                                                                 
 dense_2 (Dense)             (None, 101)               129381    
                                                                 
 activation_2 (Activation)   (None, 101)               0         
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
___________________________________________

In [ ]:
for layer in model.layers:
  print(layer, layer.dtype, layer.dtype_policy)

<keras.engine.input_layer.InputLayer object at 0x7f61ce3701f0> float32 <Policy "float32">
<keras.engine.functional.Functional object at 0x7f61cb0c4820> float32 <Policy "mixed_float16">
<keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x7f61cdc697f0> float32 <Policy "mixed_float16">
<keras.layers.core.dense.Dense object at 0x7f61cbc93a00> float32 <Policy "mixed_float16">
<keras.layers.core.activation.Activation object at 0x7f61cb13d6a0> float32 <Policy "float32">


In [ ]:
for layer in model.layers[1].layers:
  print(layer, layer.dtype, layer.dtype_policy)

<keras.engine.input_layer.InputLayer object at 0x7f61cbc93af0> float32 <Policy "float32">
<keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x7f61cbda86d0> float32 <Policy "mixed_float16">
<keras.layers.preprocessing.normalization.Normalization object at 0x7f61cc4a3910> float32 <Policy "mixed_float16">
<keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x7f61cc595b50> float32 <Policy "mixed_float16">
<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x7f61dc0cde20> float32 <Policy "mixed_float16">
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f61cca9fc40> float32 <Policy "mixed_float16">
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f61cc587040> float32 <Policy "mixed_float16">
<keras.layers.core.activation.Activation object at 0x7f61cbcbb670> float32 <Policy "mixed_float16">
<keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D object at 0x7f61cbcbb040> float32 <Policy "mixed_float16">
<k

### Fit our model

In [ ]:
model_history = model.fit(train_data,
                          epochs=5,
                          steps_per_epoch=len(train_data),
                          validation_data = test_data,
                          validation_steps=0.10*len(test_data),
                          callbacks=[ModelCheckPoint])

Epoch 1/5
2368/2368 [==============================] - ETA: 0s - loss: 1.6935 - accuracy: 0.5902

2368/2368 [==============================] - 243s 94ms/step - loss: 1.6935 - accuracy: 0.5902 - val_loss: 1.1040 - val_accuracy: 0.7037
Epoch 2/5
2368/2368 [==============================] - ETA: 0s - loss: 1.1673 - accuracy: 0.6971

2368/2368 [==============================] - 223s 93ms/step - loss: 1.1673 - accuracy: 0.6971 - val_loss: 0.9874 - val_accuracy: 0.7314
Epoch 3/5
2367/2368 [============================>.] - ETA: 0s - loss: 1.0241 - accuracy: 0.7325

2368/2368 [==============================] - 199s 83ms/step - loss: 1.0241 - accuracy: 0.7326 - val_loss: 0.9528 - val_accuracy: 0.7373
Epoch 4/5
2367/2368 [============================>.] - ETA: 0s - loss: 0.9300 - accuracy: 0.7549

2368/2368 [==============================] - 193s 80ms/step - loss: 0.9299 - accuracy: 0.7549 - val_loss: 0.9370 - val_accuracy: 0.7453
Epoch 5/5
2368/2368 [==============================] - ETA: 0s - loss: 0.8610 - accuracy: 0.7725

2368/2368 [==============================] - 205s 85ms/step - loss: 0.8610 - accuracy: 0.7725 - val_loss: 0.9127 - val_accuracy: 0.7484


In [ ]:
model.evaluate(test_data)

790/790 [==============================] - 56s 71ms/step - loss: 0.9222 - accuracy: 0.7476


[0.9222015738487244, 0.7476039528846741]

## Fine-tuning

In [ ]:
base_model.trainable=True

for layer in base_model.layers[:-10]:
  layer.trainable=False

In [ ]:
for layer in base_model.layers:
  print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f61cbc93af0> False
<keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x7f61cbda86d0> False
<keras.layers.preprocessing.normalization.Normalization object at 0x7f61cc4a3910> False
<keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x7f61cc595b50> False
<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x7f61dc0cde20> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f61cca9fc40> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f61cc587040> False
<keras.layers.core.activation.Activation object at 0x7f61cbcbb670> False
<keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D object at 0x7f61cbcbb040> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f61cbcaffa0> False
<keras.layers.core.activation.Activation object at 0x7f61cbcbef40> False
<keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(0.0001),
              metrics=['accuracy'])
model_history_fine = model.fit(train_data,
                               epochs=10,
                               steps_per_epoch=len(train_data),
                               validation_data=test_data,
                               validation_steps=0.10*len(test_data),
                               initial_epoch=model_history.epoch[-1])

Epoch 5/10
2368/2368 [==============================] - 215s 85ms/step - loss: 0.7318 - accuracy: 0.8035 - val_loss: 0.8643 - val_accuracy: 0.7638
Epoch 6/10
2368/2368 [==============================] - 192s 80ms/step - loss: 0.6030 - accuracy: 0.8393 - val_loss: 0.8654 - val_accuracy: 0.7698
Epoch 7/10
2368/2368 [==============================] - 193s 81ms/step - loss: 0.5078 - accuracy: 0.8655 - val_loss: 0.8536 - val_accuracy: 0.7718
Epoch 8/10
2368/2368 [==============================] - 194s 81ms/step - loss: 0.4265 - accuracy: 0.8888 - val_loss: 0.8690 - val_accuracy: 0.7733
Epoch 9/10
2368/2368 [==============================] - 193s 81ms/step - loss: 0.3567 - accuracy: 0.9096 - val_loss: 0.9082 - val_accuracy: 0.7631
Epoch 10/10
2368/2368 [==============================] - 205s 85ms/step - loss: 0.2956 - accuracy: 0.9274 - val_loss: 0.9251 - val_accuracy: 0.7718


In [ ]:
model.evaluate(test_data)

790/790 [==============================] - 58s 73ms/step - loss: 0.9416 - accuracy: 0.7621


[0.9416007399559021, 0.7621386051177979]